In [ ]:
nltk.do

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer
import re
import string 
from sklearn.model_selection import train_test_split
stopwords_english = stopwords.words('english')
stemmer=PorterStemmer()

LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - '/home/omkar/nltk_data'
    - '/home/omkar/anaconda3/nltk_data'
    - '/home/omkar/anaconda3/share/nltk_data'
    - '/home/omkar/anaconda3/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [ ]:
pwd

In [ ]:
train=pd.read_csv('nlp-getting-started/train.csv')

In [ ]:
train.head()

In [ ]:
x=train.text
y=train.target

In [ ]:
plt.bar(['pos','neg'],y.value_counts(),color=['r','k'])
plt.show()

In [5]:
plt.pie(y.value_counts(),labels=['pos','neg'], autopct='%1.1f%%',
        shadow=True, startangle=90)
plt.show()

NameError: name 'y' is not defined

In [26]:
def precess_tweets(tweet):
    tweet2 = re.sub(r'^RT[\s]+', '', tweet)

# remove hyperlinks
    tweet2 = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet2)

# remove hashtags
# only removing the hash # sign from the word
    tweet2 = re.sub(r'#', '', tweet2)
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)

# tokenize tweets
    tweet_tokens = tokenizer.tokenize(tweet2)
    tweet_clean=[]
    for i in tweet_tokens:
        if i not in stopwords_english and i not in string.punctuation:
            tweet_clean.append(i)
    tweet_stem=[]
    for word in tweet_clean:
        w=stemmer.stem(word)
        tweet_stem.append(w)
    return tweet_stem
        
            
            

In [34]:
def build_freq(tweets,label):
    freqs={}
    for tweet,y in zip(tweets,label):
        for i in precess_tweets(tweet):
            pair=(i,y)
            if pair in freqs:
                freqs[pair]+=1
            else:
                freqs[pair]=1
    return freqs
            

In [1]:
def fr(freqs,word,label):
    try:
        n=freqs[pair]
    except:
        n=0

In [56]:
def train_naive_bayes(freqs, train_x, train_y):
    likehood={}
    prior=0
    vocab = set([pair[0] for pair in freqs.keys()])
    V = len(vocab)
    n_pos=n_neg=0
    for pair in freqs.keys():
        if pair[1]>0:
            n_pos+=freqs[pair]
        else:
            n_neg+=freqs[pair]
    D=len(train_y)
    D_pos=(len(list(filter(lambda x: x > 0, train_y))))
    D_neg=(len(list(filter(lambda x: x <= 0, train_y))))
    prior=np.log(D_pos)-np.log(D_neg)
    for word in vocab :
        print(word)
        freq_pos=fr(freqs,word,1)
        freq_neg=fr(freqs,word,0)
        p_pos=(freq_pos+1)/(n_pos+V)
        p_neg=(freq_neg+1)/(n_neg+V)
        likehood[word]=np.log(p_pos/p_neg)
    return prior,likehood
        
            
    

In [49]:
x_train, x_test, y_train, y_test=train_test_split(x,y,test_size=0.3,random_state=42,stratify=y)

In [51]:
freqs=build_freq(x_train,y_train)


In [57]:
prior,likelihood=train_naive_bayes(freqs,x_train,y_train)

morethan


KeyError: ('morethan', 0)

In [2]:
def predict(tweet, logprior, loglikelihood):
    
    word_l = process_tweet(tweet)

    
    p = 0

    # add the logprior
    p += prior

    for word in word_l:

        # check if the word exists in the loglikelihood dictionary
        if word in llikelihood:
            # add the log likelihood of that word to the probability
            p += likelihood[word]

    ### END CODE HERE ###

    return p

In [3]:
def test_naive_bayes(x_test, y_test, prior, likelihood):
   
    accuracy = 0  # return this properly

    ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###
    y_hats = []
    for tweet in test_x:
        # if the prediction is > 0
        if predict(tweet, prior, likelihood) > 0:
            # the predicted class is 1
            y_hat_i = 1
        else:
            # otherwise the predicted class is 0
            y_hat_i = 0

        # append the predicted class to the list y_hats
        y_hats.append(y_hat_i)

    # error is the average of the absolute values of the differences between y_hats and test_y
    error = np.mean(np.absolute(y_hats-y_test))

    # Accuracy is 1 minus the error
    accuracy = 1-error

    ### END CODE HERE ###

    return accuracy

In [ ]:
print("Naive Bayes accuracy = %0.4f" %
      (test_naive_bayes(x_test, y_test, prior, likelihood)))